In [10]:
from google import genai

In [17]:
from dotenv import load_dotenv
import os

In [20]:
load_dotenv() 
API_KEY = os.getenv("GEMINI_API_KEY")
#print(API_KEY)

In [2]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

'rm' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [29]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    client = genai.Client(api_key=API_KEY)
    responses = client.models.generate_content(
        model="phi3", contents=prompt)
    return responses.text 

In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:


client = genai.Client(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
    )

In [ ]:
llm("")

ClientError: 404 NOT_FOUND. {'error': {'code': 404, 'message': 'models/phi3 is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}

In [33]:
print(_)

candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text="Boredom is a universal feeling, and the good news is there are lots of ways to combat it! The best approach depends on *why* you're bored and what resources you have available. Here's a breakdown:\n\n**1. Identify the Root Cause of Your Boredom:**\n\n*   **Lack of Stimulation:** Are you simply not being challenged or entertained enough?\n*   **Lack of Motivation:** Do you have activities you *could* do, but you just don't feel like doing them?\n*   **Routine and Monotony:** Are you stuck in the same predictable pattern day after day?\n*   **Feeling Unfulfilled:** Are you missing a sense of purpose or accomplishment?\n*   **Loneliness:** Are you feeling isolated and lacking social connection?\n*   **Mental Exhaustion:** Are you actually burnt out a